# GitHub Connector

In [1]:
import re
import github
import getpass
import pandas

## Preamble

## Connector

In [2]:


class GitHubConnector:
    def __init__(self, access_token):
        self.access_token = access_token
        self.github = github.Github(self.access_token)
        self.repo = None

    def _parse_repo_url(self, url):
        repo_pattern = r"https?://github.com/([^/]+)/(.+)"
        match = re.match(repo_pattern, url)
        if match:
            return match.groups()
        else:
            return None
        
    def connect_repo(self, repo_url):
        """ Connect to a GitHub repository"""
        repo_info = self._parse_repo_url(repo_url)
        if repo_info:
            owner, repo_name = repo_info
            print(repo_info)
        else:
            print("Invalid repository URL")
            return

        # Get the repository object
        self.repo = self.github.get_repo(f"{owner}/{repo_name}")
        return self


    def get_issues(self, state="all"):
        if self.repo is None:
            print("Please connect to a repository first")
            return

        # Retrieve all issues
        issues = self.repo.get_issues(state=state)

        return list(issues)
    

    def get_issue_events(self, issue):
        if self.repo is None:
            print("Repository not set")
            return

        if isinstance(issue, int):
            issue_number = issue
            issue = self.repo.get_issue(issue_number)

        comments = list(issue.get_comments())
        events = list(issue.get_events())

        return comments, events


In [3]:
# Example usage
access_token = getpass.getpass("Enter your GitHub access token: ")
github_connector = GitHubConnector(access_token)


In [4]:
url = input("Enter the repository URL: ")
github_connector.connect_repo(url)
issues = github_connector.get_issues(state="all")


('tuttle-dev', 'tuttle')


In [5]:
issues

[Issue(title="View: display project progress", number=222),
 Issue(title="Bug: Invoice number not incrementing", number=221),
 Issue(title="Cascading delete for invoice and timesheet items", number=220),
 Issue(title="Dev delete integrity", number=219),
 Issue(title="Test: Integration test for successful app start", number=218),
 Issue(title="Python packaging", number=217),
 Issue(title="Distribution: Refactor into functioning Python package", number=216),
 Issue(title="Data integrity: Prevent deletion of required entities", number=215),
 Issue(title="UI: Sidebar layout", number=214),
 Issue(title="Test: Time tracking from Toggl", number=213),
 Issue(title="UI: color-coded message boxes for snack bar", number=212),
 Issue(title="feat: added a confirmation step during app reset", number=211),
 Issue(title="Ui/Model: Add field validation / format methods", number=210),
 Issue(title="UI: Place form field errors on dialogs below related field", number=209),
 Issue(title="refactor: prefix c

In [6]:
an_issue = issues[0]

In [7]:
def issues_to_dataframe(issues):
    """ Convert a list of issues to a pandas DataFrame"""
    data = []

    for issue in issues:
        issue_data = {
            "number": issue.number,
            "title": issue.title,
            "state": issue.state,
            "author": issue.user.login,
            "created_at": issue.created_at,
            "updated_at": issue.updated_at,
            "closed_at": issue.closed_at,
            "labels": [label.name for label in issue.labels],
            "comments": issue.comments,
            "html_url": issue.html_url,
            "body": issue.body
        }
        data.append(issue_data)

    df = pandas.DataFrame(data)
    return df



In [8]:
issues_data = issues_to_dataframe(issues)
issues_data

,number,title,state,author,created_at,updated_at,closed_at,labels,comments,html_url,body
0,222,View: display project progress,open,clstaudt,2023-03-12 11:35:44,2023-03-17 11:17:49,NaT,"[enhancement, help wanted]",0,https://github.com/tuttle-dev/tuttle/issues/222,As a user I want to see at a glance how much o...
1,221,Bug: Invoice number not incrementing,closed,clstaudt,2023-02-09 20:42:11,2023-02-09 21:35:33,2023-02-09 21:35:33,"[bug, backend]",0,https://github.com/tuttle-dev/tuttle/issues/221,"For invoices created on the same date, the inv..."
2,220,Cascading delete for invoice and timesheet items,closed,clstaudt,2023-02-09 20:35:33,2023-02-09 20:37:23,2023-02-09 20:37:18,[],0,https://github.com/tuttle-dev/tuttle/pull/220,None
3,219,Dev delete integrity,open,clstaudt,2023-02-09 14:25:24,2023-02-09 15:45:22,NaT,[],0,https://github.com/tuttle-dev/tuttle/pull/219,Ensure data model integrity by preventing dele...
4,218,Test: Integration test for successful app start,open,clstaudt,2023-02-09 14:13:49,2023-02-11 05:29:24,NaT,"[good first issue, help wanted, distribution]",12,https://github.com/tuttle-dev/tuttle/issues/218,First attempt: This is not yet working since ...
...,...,...,...,...,...,...,...,...,...,...,...
209,7,Account Overview (List),open,clstaudt,2021-10-27 08:41:30,2021-10-27 08:41:44,NaT,"[enhancement, good first issue]",0,https://github.com/tuttle-dev/tuttle/issues/7,"As a user, I want to be able to add all my bus..."
210,6,Effective Salary,open,clstaudt,2021-10-27 08:39:37,2021-10-30 14:08:44,NaT,[enhancement],2,https://github.com/tuttle-dev/tuttle/issues/6,"As a freelancer, I want to pay out an effectiv..."
211,5,Account Balance Overview,open,clstaudt,2021-10-27 08:34:07,2021-10-27 08:35:29,NaT,"[enhancement, good first issue]",1,https://github.com/tuttle-dev/tuttle/issues/5,"As a user, I want to see the balances of all m..."
212,4,Anonymize Clients for Cloud Connectors,open,clstaudt,2021-10-27 08:30:05,2021-10-27 08:30:22,NaT,[enhancement],0,https://github.com/tuttle-dev/tuttle/issues/4,"As a freelancer, I want the option to anonymiz..."


In [9]:
good_first_issues_data = issues_data[issues_data['labels'].apply(lambda labels: 'good first issue' in labels)]
good_first_issues_data


,number,title,state,author,created_at,updated_at,closed_at,labels,comments,html_url,body
4,218,Test: Integration test for successful app start,open,clstaudt,2023-02-09 14:13:49,2023-02-11 05:29:24,NaT,"[good first issue, help wanted, distribution]",12,https://github.com/tuttle-dev/tuttle/issues/218,First attempt: This is not yet working since ...
8,214,UI: Sidebar layout,open,clstaudt,2023-02-01 22:27:05,2023-02-28 13:12:24,NaT,"[good first issue, UX/UI]",9,https://github.com/tuttle-dev/tuttle/issues/214,\n- [ ] The width of the side menu bar current...
10,212,UI: color-coded message boxes for snack bar,open,clstaudt,2023-01-30 18:02:44,2023-01-30 18:03:09,NaT,"[enhancement, good first issue, help wanted, U...",0,https://github.com/tuttle-dev/tuttle/issues/212,A cool UI feature would be to color-code the s...
20,201,UI/Model: Split User.name into first_name and ...,closed,clstaudt,2023-01-28 10:24:25,2023-02-09 08:32:10,2023-02-09 08:32:09,"[good first issue, backend, UX/UI]",6,https://github.com/tuttle-dev/tuttle/issues/201,Test feedback:\n\n> Name: It might be helpful ...
30,190,Feature: Contract documents can be attached to...,open,clstaudt,2023-01-24 10:14:49,2023-01-24 10:16:19,NaT,"[enhancement, good first issue, help wanted]",0,https://github.com/tuttle-dev/tuttle/issues/190,As a freelancer I want the app to store and he...
32,188,"UI: sorting for projects, clients, contracts",open,clstaudt,2023-01-24 09:10:01,2023-01-24 09:11:40,NaT,"[enhancement, good first issue, help wanted, U...",0,https://github.com/tuttle-dev/tuttle/issues/188,"If a user has added many business data items, ..."
38,182,UI: registration / user profile: ask user for ...,closed,clstaudt,2023-01-23 17:16:52,2023-01-24 08:16:07,2023-01-24 08:16:06,"[bug, good first issue, UX/UI]",3,https://github.com/tuttle-dev/tuttle/issues/182,```python\n website: Optional[str] = Field(...
42,178,UI & Backend: Get banking information for user,closed,clstaudt,2023-01-22 18:44:41,2023-01-24 08:33:15,2023-01-24 08:33:15,"[good first issue, help wanted, backend, UX/UI]",1,https://github.com/tuttle-dev/tuttle/issues/178,None
44,176,Invoicing: note about reverse charge,open,clstaudt,2023-01-22 18:34:39,2023-01-22 18:36:19,NaT,"[enhancement, good first issue, help wanted]",0,https://github.com/tuttle-dev/tuttle/issues/176,"> If you ask VAT, for billing purposes you mig..."
45,175,UX: simplified invoicing without time tracking,open,clstaudt,2023-01-22 18:30:48,2023-01-22 18:31:11,NaT,"[enhancement, good first issue, help wanted, b...",0,https://github.com/tuttle-dev/tuttle/issues/175,> I think it would be nice if I could just man...


In [10]:
an_issue

Issue(title="View: display project progress", number=222)

In [12]:
comments, events = github_connector.get_issue_events(an_issue)

In [13]:
events

[IssueEvent(id=8725495098),
 IssueEvent(id=8725495101),
 IssueEvent(id=8777175292),
 IssueEvent(id=8777175381)]